In [1]:
# breakhistf2v16.ipynb  v1.0.0 
# 
#  --------------------------------------------------
#  Hangzhou Domain Zones Technology Co., Ltd

#  Apache Licence 2.0       https://www.apache.org/licenses/LICENSE-2.0
#  --------------------------------------------------

import os
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow import keras
import neural_structured_learning as nsl
from tensorflow.python.keras.api._v2.keras import layers, optimizers, losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')
  
%matplotlib inline

def read_csv(csvnamepath, filename):
     

    # read from csv file
    images, labels = [], []
    with open(os.path.join(csvnamepath, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)

            images.append(img)
            labels.append(label)

    assert len(images) == len(labels)

    return images, labels

 
def load_breakhis(dirc, mode='train'):
    classdir2label={}
    filename="tf2breakhisCSV"
    filedirs = os.listdir( dirc)
    for filedir in filedirs:
        if not os.path.isdir(os.path.join(dirc,filedir)):
            continue
        classdir2label[filedir]=len(classdir2label.keys())
    

     
    images, labels = read_csv(os.path.join(os.path.abspath('.'),'tf2breakhis'), filename )  
    if mode == 'train':  # 60%
        images = images[:int(0.6 * len(images))]
        labels = labels[:int(0.6 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # 20% = 80%->100%
        images = images[int(0.8 * len(images)):]
        labels = labels[int(0.8 * len(labels)):]
    return images, labels, classdir2label
 
img_mean = tf.constant([0.485, 0.456, 0.406])
img_std = tf.constant([0.229, 0.224, 0.225])

@tf.function
def normalize(x, mean=img_mean, std=img_std):
    # x shape: [224, 224, 3]
    # mean：shape为1；这里用到了广播机制。我们安装好右边对齐的原则，可以得到如下；
    # mean : [1, 1, 3], std: [3]        先插入1
    # mean : [224, 224, 3], std: [3]    再变为224
    x = (x - mean)/std
    return x

# 数据normalize之后，这里有一个反normalizaion操作。比如数据可视化的时候，需要反过来。
@tf.function
def denormalize(x, mean=img_mean, std=img_std):
    x = x * std + mean
    return x

 


# 预处理的函数，复制过来。
@tf.function
def preprocess(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
   
    x = tf.image.random_flip_left_right(x)
    #plt.imshow(x)
  
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
    #plt.imshow(x)
    
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    # x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y


# 预处理的函数，复制过来。
@tf.function
def preprocess1(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
    
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
      
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    # x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y




 

batchsz = 64
epochnum =10
# creat train db   一般训练的时候需要shuffle。其它是不需要的。
images, labels, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'train')
db_train = tf.data.Dataset.from_tensor_slices((images, labels))  # 变成个Dataset对象。
db_train = db_train.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)  # map函数图片路径变为内容。
 

# crate validation db
images2, labels2, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'val')
db_val = tf.data.Dataset.from_tensor_slices((images2, labels2))
db_val = db_val.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)
# create test db
images3, labels3, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'test')
db_test = tf.data.Dataset.from_tensor_slices((images3, labels3))
db_test = db_test.map(preprocess1).batch(batchsz)

 
resnetdense = keras.Sequential([
    layers.Conv2D(16,5,3),
    layers.MaxPool2D(3,3),
    layers.ReLU(),
    layers.Conv2D(64,5,3),
    layers.MaxPool2D(2,2),
    layers.ReLU(),
    
    layers.Flatten(),
    #
    layers.Dense(128), 
    layers.Dropout(rate=0.5),
    
    
    
    layers.Dense(64),  
    layers.ReLU(),
    layers.Dense(8)
   ])


 

# 首先创建Resnet18
# resnet = ResNet(8)

resnetdense.build(input_shape=(batchsz, 224, 224, 3))
resnetdense.summary()

# monitor监听器, 连续5个验证准确率不增加，这个事情触发。
 
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=100

)

  

 
# 网络的装配。
resnetdense.compile(optimizer=optimizers.Adam(lr=1e-4),
               loss=losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

# 完成标准的train，val, test;
# 标准的逻辑必须通过db_val挑选模型的参数，就需要提供一个earlystopping技术，

LOGDIR='log/breakhistf2v16' 
resnetdense.fit(db_train, validation_data=db_val, 
                        validation_freq=10, epochs=700, 
                        callbacks=[TensorBoard(log_dir=LOGDIR)])   # 1个epoch验证1次。触发了这个事情，提前停止了。
resnetdense.evaluate(db_test)
 
 

    
    
 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  25664     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0         
_________________________________________________________________
flatten (Flatten)            multiple                  0

149/149 [==============================] - 191s 1s/step - loss: 0.8937 - accuracy: 0.6699
Epoch 136/700
149/149 [==============================] - 196s 1s/step - loss: 0.8940 - accuracy: 0.6717
Epoch 137/700
149/149 [==============================] - 192s 1s/step - loss: 0.8987 - accuracy: 0.6701
Epoch 138/700
149/149 [==============================] - 196s 1s/step - loss: 0.9028 - accuracy: 0.6685
Epoch 139/700
149/149 [==============================] - 193s 1s/step - loss: 0.8920 - accuracy: 0.6701
Epoch 140/700
149/149 [==============================] - 259s 2s/step - loss: 0.8877 - accuracy: 0.6767 - val_loss: 0.9569 - val_accuracy: 0.6448
Epoch 141/700
149/149 [==============================] - 192s 1s/step - loss: 0.9012 - accuracy: 0.6694
Epoch 142/700
149/149 [==============================] - 196s 1s/step - loss: 0.8727 - accuracy: 0.6806
Epoch 143/700
149/149 [==============================] - 192s 1s/step - loss: 0.8870 - accuracy: 0.6743
Epoch 144/700
149/149 [=============

Epoch 211/700
149/149 [==============================] - 196s 1s/step - loss: 0.7630 - accuracy: 0.7157
Epoch 212/700
149/149 [==============================] - 192s 1s/step - loss: 0.7655 - accuracy: 0.7118
Epoch 213/700
149/149 [==============================] - 190s 1s/step - loss: 0.7704 - accuracy: 0.7107
Epoch 214/700
149/149 [==============================] - 197s 1s/step - loss: 0.7599 - accuracy: 0.7208
Epoch 215/700
149/149 [==============================] - 197s 1s/step - loss: 0.7595 - accuracy: 0.7210
Epoch 216/700
149/149 [==============================] - 191s 1s/step - loss: 0.7566 - accuracy: 0.7209
Epoch 217/700
149/149 [==============================] - 194s 1s/step - loss: 0.7560 - accuracy: 0.7205
Epoch 218/700
149/149 [==============================] - 193s 1s/step - loss: 0.7552 - accuracy: 0.7172
Epoch 219/700
149/149 [==============================] - 195s 1s/step - loss: 0.7511 - accuracy: 0.7217
Epoch 220/700
149/149 [==============================] - 250s 2s

149/149 [==============================] - 180s 1s/step - loss: 0.6092 - accuracy: 0.7673
Epoch 362/700
149/149 [==============================] - 180s 1s/step - loss: 0.5962 - accuracy: 0.7714
Epoch 363/700
149/149 [==============================] - 178s 1s/step - loss: 0.5867 - accuracy: 0.7743
Epoch 364/700
149/149 [==============================] - 178s 1s/step - loss: 0.5981 - accuracy: 0.7705
Epoch 365/700
149/149 [==============================] - 179s 1s/step - loss: 0.6036 - accuracy: 0.7663
Epoch 366/700
149/149 [==============================] - 176s 1s/step - loss: 0.6143 - accuracy: 0.7704
Epoch 367/700
149/149 [==============================] - 176s 1s/step - loss: 0.5961 - accuracy: 0.7705
Epoch 368/700
149/149 [==============================] - 176s 1s/step - loss: 0.6050 - accuracy: 0.7684
Epoch 369/700
149/149 [==============================] - 176s 1s/step - loss: 0.6073 - accuracy: 0.7635
Epoch 370/700
149/149 [==============================] - 235s 2s/step - loss: 

149/149 [==============================] - 179s 1s/step - loss: 0.5497 - accuracy: 0.7865
Epoch 438/700
149/149 [==============================] - 180s 1s/step - loss: 0.5412 - accuracy: 0.7909
Epoch 439/700
149/149 [==============================] - 176s 1s/step - loss: 0.5535 - accuracy: 0.7876
Epoch 440/700
149/149 [==============================] - 235s 2s/step - loss: 0.5544 - accuracy: 0.7825 - val_loss: 0.7359 - val_accuracy: 0.7225
Epoch 441/700
149/149 [==============================] - 202s 1s/step - loss: 0.5354 - accuracy: 0.7952
Epoch 442/700
149/149 [==============================] - 197s 1s/step - loss: 0.5411 - accuracy: 0.7901
Epoch 443/700
149/149 [==============================] - 193s 1s/step - loss: 0.5232 - accuracy: 0.7953
Epoch 444/700
149/149 [==============================] - 194s 1s/step - loss: 0.5457 - accuracy: 0.7841
Epoch 445/700
149/149 [==============================] - 193s 1s/step - loss: 0.5295 - accuracy: 0.7976
Epoch 446/700
149/149 [=============

149/149 [==============================] - 192s 1s/step - loss: 0.4600 - accuracy: 0.8171
Epoch 589/700
149/149 [==============================] - 192s 1s/step - loss: 0.4554 - accuracy: 0.8198
Epoch 590/700
149/149 [==============================] - 251s 2s/step - loss: 0.4602 - accuracy: 0.8220 - val_loss: 0.6721 - val_accuracy: 0.7538
Epoch 591/700
149/149 [==============================] - 192s 1s/step - loss: 0.4603 - accuracy: 0.8161
Epoch 592/700
149/149 [==============================] - 193s 1s/step - loss: 0.4585 - accuracy: 0.8231
Epoch 593/700
149/149 [==============================] - 193s 1s/step - loss: 0.4622 - accuracy: 0.8166
Epoch 594/700
149/149 [==============================] - 192s 1s/step - loss: 0.4604 - accuracy: 0.8187
Epoch 595/700
149/149 [==============================] - 190s 1s/step - loss: 0.4635 - accuracy: 0.8162
Epoch 596/700
149/149 [==============================] - 194s 1s/step - loss: 0.4378 - accuracy: 0.8258
Epoch 597/700
149/149 [=============

[0.728386754989624, 0.7357775]